## Quantum Programming Project - Modular Exponentiation

Project-03

- Lea Jesenkovic
- Lars Herbold

In [ ]:
from qiskit import QuantumCircuit
from logic_gates import LogicGatesExtension    

### MONKEY PATCHING ###
QuantumCircuit.and_gate = LogicGatesExtension.apply_and
QuantumCircuit.or_gate = LogicGatesExtension.apply_or
QuantumCircuit.xor_gate = LogicGatesExtension.apply_xor
QuantumCircuit.controlled_and_gate = LogicGatesExtension.apply_controlled_and
QuantumCircuit.controlled_or_gate = LogicGatesExtension.apply_controlled_or
QuantumCircuit.controlled_xor_gate = LogicGatesExtension.apply_controlled_xor

def set_bits(circuit, A, X):
    """
    Initializes the bits of register A with the binary string X.
    - circuit: The QuantumCircuit object
    - A: List of qubit indices
    - X: Binary string (e.g., '01011')
    """
    for i, bit in enumerate(X):
        if bit == '1':
            circuit.x(A[i])

def copy(circuit, A, B):
    """
    Copies the binary string from register A to register B.
    Assumes len(A) == len(B) and B is initialized to |0>.
    """
    for i in range(len(A)):
        # Apply CNOT: control is A[i], target is B[i]
        circuit.cx(A[i], B[i])

def controlled_copy(circuit, c, A, B):
    """
    If c=1: B ^= A  (bitwise controlled copy)
    If c=0: do nothing
    """
    for i in range(len(A)):
        circuit.ccx(c, A[i], B[i])

def controlled_not(circuit, c, q):
    """If c=1: apply X to q."""
    circuit.cx(c, q)


def full_adder(circuit, a, b, r, c_in, c_out, AUX):
    # Compute a ⊕ b into AUX[2]
    circuit.xor_gate(a, b, AUX[2])

    # r = AUX[2] ⊕ c_in
    circuit.xor_gate(AUX[2], c_in, r)
    # circuit.cx(AUX[2], r)
    # circuit.cx(c_in, r)

    # c_out parts
    circuit.and_gate(AUX[2], c_in, AUX[0])
    circuit.and_gate(a, b, AUX[1])
    circuit.or_gate(AUX[0], AUX[1], c_out)

    # CLEANUP (reverse order)
    circuit.and_gate(a, b, AUX[1])
    circuit.and_gate(AUX[2], c_in, AUX[0])
    circuit.xor_gate(a, b, AUX[2])

def controlled_full_adder(circuit, c, a, b, r, c_in, c_out, AUX):
    """
    Controlled full adder.
    If c=1:
        r     ^= a ⊕ b ⊕ c_in
        c_out ^= (a & b) | (c_in & (a ⊕ b))
    If c=0:
        no change

    AUX layout:
      AUX[0], AUX[1], AUX[2] are clean ancillas (start |0>) used internally.
      We will use:
        t = AUX[2] as (a ⊕ b)
        u0 = AUX[0] as (t & c_in) term
        u1 = AUX[1] as (a & b) term
    Assumes r and c_out start |0> if you want them to become the outputs.
    """
    u0 = AUX[0]
    u1 = AUX[1]
    t  = AUX[2]

    # t := a ⊕ b
    circuit.xor_gate(a, b, t)

    # r ^= (a ⊕ b) and r ^= c_in  (only when c=1)
    circuit.ccx(c, t, r)        # controlled copy of t into r
    circuit.ccx(c, c_in, r)     # controlled copy of c_in into r

    # u0 := c ? (t & c_in)
    circuit.controlled_and_gate(c, t, c_in, u0)

    # u1 := c ? (a & b)
    circuit.controlled_and_gate(c, a, b, u1)

    # c_out ^= c ? (u0 | u1)
    circuit.controlled_or_gate(c, u0, u1, c_out)

    # CLEANUP (reverse)
    circuit.controlled_and_gate(c, a, b, u1)
    circuit.controlled_and_gate(c, t, c_in, u0)
    circuit.xor_gate(a, b, t)


def add(circuit, A, B, R, AUX):
    """
    Adds number(A) to number(B) and stores the result in register R.
    A, B, R: Lists of qubit indices.
    AUX: List of qubit indices for carry bits and internal full_adder logic.
    """
    n = len(A)
    
    # We need a carry-in (c_in) and a carry-out (c_out) for each bit.
    # For bit 'i', c_in is AUX[i] and c_out is AUX[i+1].
    # Let's reserve the first few qubits of AUX for carries and 
    # the rest for the full_adder's internal temporary workspace.
    carries = AUX[:n+1] 
    fa_internal_aux = AUX[n+1:n+1+3] 

    for i in range(n):
        # Apply full_adder for each bit position
        # inputs: A[i], B[i], carries[i] (c_in)
        # outputs: R[i] (result), carries[i+1] (c_out)
        full_adder(
            circuit,
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

def controlled_add(circuit, c, A, B, R, AUX):
    """
    If c=1: R ^= (A + B)
    If c=0: R unchanged

    Uses ripple-carry of controlled_full_adder.
    AUX layout:
      carries: AUX[0 : n+1]
      fa_aux:  AUX[n+1 : n+1+3]  (3 clean ancillas used by controlled_full_adder)
    """
    n = len(A)
    carries = AUX[:n+1]
    fa_aux  = AUX[n+1:n+1+3]

    for i in range(n):
        controlled_full_adder(circuit, c, A[i], B[i], R[i], carries[i], carries[i+1], fa_aux)


def subtract(circuit, A, B, R, AUX):
    """
    Subtracts Number(B) from Number(A) and stores the result in R[cite: 107].
    Logic: A - B = A + (NOT B) + 1.
    """
    n = len(A)
    carries = AUX[:n+1]
    fa_internal_aux = AUX[n+1:n+1+3]

    # 1. Negate each bit in B 
    for b_qubit in B:
        circuit.x(b_qubit)

    # 2. Set the first carry-in bit to 1 
    circuit.x(carries[0])

    # 3. Apply the adder circuit cascade 
    for i in range(n):
        full_adder(
            circuit,
            A[i], 
            B[i], 
            R[i], 
            carries[i], 
            carries[i+1], 
            fa_internal_aux
        )

    # 4. CLEANUP: Reset B and the first carry-in to original state
    # (Note: AUX carries are usually uncomputed in modular steps)
    circuit.x(carries[0])
    for b_qubit in B:
        circuit.x(b_qubit)

def controlled_subtract(circuit, c, A, B, R, AUX):
    """
    If c=1: R ^= (A - B)
    If c=0: R unchanged

    Implements A - B = A + (~B) + 1, but all steps are controlled by c.
    AUX layout:
      carries: AUX[0 : n+1]
      fa_aux:  AUX[n+1 : n+1+3]
    """
    n = len(A)
    carries = AUX[:n+1]
    fa_aux  = AUX[n+1:n+1+3]

    # Controlled negate B (only if c=1)
    for qb in B:
        controlled_not(circuit, c, qb)

    # Controlled set carry-in = 1 (only if c=1)
    controlled_not(circuit, c, carries[0])

    # Controlled ripple add: A + B + carry
    for i in range(n):
        controlled_full_adder(circuit, c, A[i], B[i], R[i], carries[i], carries[i+1], fa_aux)

    # Cleanup: undo carry-in and undo B negation (only if c=1)
    controlled_not(circuit, c, carries[0])
    for qb in B:
        controlled_not(circuit, c, qb)

def greater_or_eq(circuit, A, B, r, AUX):
    """
    Tests whether A >= B and stores result in r[cite: 141].
    """
    n = len(A)
    # R must be initialized to |0> for the subtraction result
    # We can use a part of AUX as a temporary R register
    temp_R = AUX[n+4 : n+4+n] 
    
    # 1. Perform subtraction
    subtract(circuit, A, B, temp_R, AUX)
    
    # 2. The final carry bit (AUX[n]) determines the result 
    circuit.cx(AUX[n], r)
    
    # 3. CLEANUP: Uncompute subtraction to reset AUX and temp_R to |0>
    subtract(circuit, A, B, temp_R, AUX)

def controlled_greater_or_eq(circuit, c, A, B, r, AUX):
    """
    If c=1: r ^= [A >= B]
    If c=0: r unchanged

    Compute flag into tmp bit, controlled-copy into r, uncompute.
    AUX layout:
      tmp_flag: uses AUX[0]
      rest:     AUX[1:] for greater_or_eq
    """
    tmp_flag = AUX[0]
    rest = AUX[1:]

    greater_or_eq(circuit, A, B, tmp_flag, rest)
    circuit.ccx(c, tmp_flag, r)
    greater_or_eq(circuit, A, B, tmp_flag, rest)

# TEST greater_or_eq
qc = QuantumCircuit(4 + 40)
A = [0,1]
B = [2,3]
r = 4
AUX = list(range(5, 45))

set_bits(qc, A, "10")
set_bits(qc, B, "01")

greater_or_eq(qc, A, B, r, AUX)
print(qc.draw(fold=120))

def add_mod(circuit, N, A, B, R, AUX):
    """
    Computes R := (A + B) mod N.

    Assumptions:
      - len(A)=len(B)=len(N)=len(R)=n
      - A,B < N
      - R starts in |0>
      - AUX starts in |0> and must end in |0>
      - Your add/subtract/greater_or_eq are correct and AUX-clean.

    AUX layout needed:
      - SUM:  n qubits
      - DIFF: n qubits
      - flag: 1 qubit
      - rest: scratch for add/sub/greater_or_eq
    """
    n = len(A)

    SUM  = AUX[0:n]           # will hold S = A + B
    DIFF = AUX[n:2*n]         # will hold D = S - N
    flag = AUX[2*n]           # 1 if S >= N else 0
    aux_rest = AUX[2*n+1:]    # scratch for add/sub/greater_or_eq

    # --- helpers: controlled copy using Toffoli ---
    def controlled_copy(ctrl, SRC, DST):
        # if ctrl==1: DST ^= SRC (assuming DST starts 0 -> DST = SRC)
        for i in range(len(SRC)):
            circuit.ccx(ctrl, SRC[i], DST[i])

    # 1) SUM = A + B
    add(circuit, A, B, SUM, aux_rest)

    # 2) flag = (SUM >= N)
    greater_or_eq(circuit, SUM, N, flag, aux_rest)

    # 3) DIFF = SUM - N
    subtract(circuit, SUM, N, DIFF, aux_rest)

    # 4) Write result to R:
    #    if flag==0 -> R = SUM
    #    if flag==1 -> R = DIFF
    #
    # R is assumed to start at 0.
    circuit.x(flag)                 # now flag = NOT flag
    controlled_copy(flag, SUM, R)   # if original flag was 0, copy SUM into R
    circuit.x(flag)                 # restore flag

    controlled_copy(flag, DIFF, R)  # if original flag was 1, copy DIFF into R

    # 5) Uncompute DIFF back to 0
    subtract(circuit, SUM, N, DIFF, aux_rest)

    # 6) Uncompute flag back to 0
    greater_or_eq(circuit, SUM, N, flag, aux_rest)

    # 7) Uncompute SUM back to 0
    add(circuit, A, B, SUM, aux_rest)

def controlled_add_mod(circuit, c, N, A, B, R, AUX):
    """
    If c=1: R ^= (A + B) mod N
    If c=0: R unchanged

    Uses compute-copy-uncompute with your existing add_mod.
    AUX layout:
      tmp:  n bits
      rest: scratch for add_mod
    """
    n = len(A)
    tmp  = AUX[:n]
    rest = AUX[n:]

    add_mod(circuit, N, A, B, tmp, rest)
    controlled_copy(circuit, c, tmp, R)
    add_mod(circuit, N, A, B, tmp, rest)

def times_two_mod(circuit, N, A, R, AUX):
    """
    Computes R := (2 * A) mod N.

    Assumptions:
      - len(A) = len(N) = len(R) = n
      - A < N
      - R starts in |0>
      - AUX starts in |0> and must end in |0>
      - add_mod is correct and AUX-clean
    """
    n = len(A)

    # Use part of AUX as a temporary copy of A
    A_copy = AUX[:n]
    aux_rest = AUX[n:]

    # 1) Copy A into A_copy
    copy(circuit, A, A_copy)

    # 2) Compute (A + A) mod N into R
    add_mod(circuit, N, A, A_copy, R, aux_rest)

    # 3) Uncompute A_copy back to |0>
    copy(circuit, A, A_copy)

def controlled_times_two_mod(circuit, c, N, A, R, AUX):
    """
    If c=1: R ^= (2*A) mod N
    If c=0: R unchanged
    """
    n = len(A)
    tmp  = AUX[:n]
    rest = AUX[n:]

    times_two_mod(circuit, N, A, tmp, rest)
    controlled_copy(circuit, c, tmp, R)
    times_two_mod(circuit, N, A, tmp, rest)

# TEST times_two_mod
# qc = QuantumCircuit(8 + 30)

# N   = [0, 1]
# A   = [2, 3]
# R   = [4, 5]
# AUX = list(range(6, 36))

# set_bits(qc, N, "11")   # N = 3
# set_bits(qc, A, "10")   # A = 2

# times_two_mod(qc, N, A, R, AUX)

# print(qc.draw())

def times_two_power_mod(circuit, N, A, k, R, AUX):
    """
    Computes R := (2^k * A) mod N.

    Assumptions:
      - len(A)=len(N)=len(R)=n
      - R starts in |0>
      - AUX starts in |0> and must end in |0>
      - add_mod is correct and AUX-clean
      - k is a small classical integer

    AUX requirement:
      - (k+1) blocks of n qubits to store V0..Vk
      - plus extra scratch qubits for add_mod (same scratch reused each call)
    """
    n = len(A)

    # allocate chain registers
    blocks_needed = (k + 1) * n
    V_all = AUX[:blocks_needed]
    scratch = AUX[blocks_needed:]

    V = [V_all[i*n:(i+1)*n] for i in range(k+1)]

    # V0 := A
    copy(circuit, A, V[0])

    # Forward: Vi := 2*V(i-1) mod N
    for i in range(1, k+1):
        times_two_mod(circuit, N, V[i-1], V[i], scratch)

    # Copy Vk to R
    copy(circuit, V[k], R)

    # Uncompute backward: clear Vk..V1
    for i in range(k, 0, -1):
        times_two_mod(circuit, N, V[i-1], V[i], scratch)

    # Clear V0
    copy(circuit, A, V[0])

def controlled_times_two_power_mod(circuit, c, N, A, k, R, AUX):
    """
    If c=1: R ^= (2^k * A) mod N
    If c=0: R unchanged
    """
    n = len(A)
    tmp  = AUX[:n]
    rest = AUX[n:]

    times_two_power_mod(circuit, N, A, k, tmp, rest)
    controlled_copy(circuit, c, tmp, R)
    times_two_power_mod(circuit, N, A, k, tmp, rest)


# TEST times_two_power_mod
# qc = QuantumCircuit(8 + 60)   # 8 data qubits + lots of AUX

# N   = [0, 1]
# A   = [2, 3]
# R   = [4, 5]
# AUX = list(range(6, 68))      # 62 AUX qubits

# set_bits(qc, N, "11")   # N = 3
# set_bits(qc, A, "01")   # A = 1

# times_two_power_mod(qc, N, A, k=2, R=R, AUX=AUX)

# print(qc.draw(fold=120))








      ┌───┐                                                                                                         »
 q_0: ┤ X ├───────■─────────■─────────────────────────────■─────────■───────────────────────────────────────────────»
      └───┘       │         │                             │         │                                               »
 q_1: ────────────┼─────────┼─────────────────────────────┼─────────┼───────────────────■─────────■─────────────────»
      ┌───┐       │         │                             │         │            ┌───┐  │  ┌───┐  │                 »
 q_2: ┤ X ├───────┼────■────■─────────────────────────────■─────────┼─────────■──┤ X ├──┼──┤ X ├──┼─────────────────»
      ├───┤┌───┐  │    │    │                             │         │         │  └───┘  │  └───┘  │                 »
 q_3: ┤ X ├┤ X ├──┼────┼────┼─────────────────────────────┼─────────┼─────────┼─────────┼────■────■─────────────────»
      └───┘└───┘  │    │    │                           